In [36]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import pickle

# Load the model and encoders
model = load_model('model.h5')

with open('geo_encoders.pkl', 'rb') as f:
    encoder_geo = pickle.load(f)

with open('gender_encoders.pkl', 'rb') as f:
    encoder_gender = pickle.load(f)

with open('scaler_df.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Sample input
Input_data = {
    'CreditScore': 0,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 42,
    'Tenure': 2,
    'Balance': 0.00,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 101348.88,
    'Exited': 1
}

# Convert input to DataFrame
input_df = pd.DataFrame([Input_data])

# Encode Geography (One-Hot Encoding)
geo_one_hot = encoder_geo.transform(input_df[['Geography']]).toarray()
geo_one_hot_df = pd.DataFrame(geo_one_hot, columns=encoder_geo.get_feature_names_out(['Geography']))

# Encode Gender (Label Encoding)
input_df['Gender'] = encoder_gender.transform(input_df['Gender'])

# Merge one-hot encoded Geography and drop original column
input_df = pd.concat([input_df.drop(columns=['Geography']), geo_one_hot_df], axis=1)

# Ensure all features match those used during training
missing_cols = set(scaler.feature_names_in_) - set(input_df.columns)
for col in missing_cols:
    input_df[col] = 0  # Add missing columns with default value 0

# Reorder columns to match scaler training data
input_df = input_df[scaler.feature_names_in_]

# Scale the data
input_scaled = scaler.transform(input_df)

print("Input Scaled Successfully!")


Input Scaled Successfully!


In [37]:
input_scaled

array([[-6.70646903, -1.09168714,  0.30102557, -1.031415  , -1.21571749,
        -0.92159124,  0.64259497,  0.9687384 ,  0.02045017,  0.98560362,
        -0.5698444 , -0.57369368]])

In [38]:
# Prediction
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step


array([[0.47541264]], dtype=float32)

In [40]:
prediction_proba=prediction[0][0]
prediction_proba

0.47541264

In [41]:
if prediction_proba>0.5:
    print("The customer is likely to churn")
else:
    print("The customer is likely to not churn")

The customer is likely to not churn
